In [219]:
import pandas as pd
# pip install python-Levenshtein for fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import numpy as np
from pyvis.network import Network
from pyvis import network as net

In [149]:
# import the data file you generated & the languages.csv datafile from wals fork
data = pd.read_csv("compiled_data.csv")
# rename the Language column to Name for merging
data = data.rename(columns={'Language': 'Name'})

wals_data = pd.read_csv("languages.csv")


In [150]:
# join by Name column, will populate lang family information if match
merged_df = pd.merge(data, wals_data, on='Name', how='left')


# Estimating typing/spelling errors using Levenshtein distance
- Humans are prone to error and typeos can occur when writing out name. We can use the package, fuzzywuzzy, to try to identify these mistakes. 
- There are types of ratios you can estimate these differences with:


In [155]:
# simple ratio 
fuzz.ratio("this is a test", "this is a test!")


97

In [154]:
# partial ratio 
fuzz.partial_ratio("this is a test", "this is a test!") 

86

Most likely we are going to care about the surname, as there are lots of alternate ways to spell first names (and overall, first names are less identifiable). To do this, we can start by extracting the surname column and make a list of the author names that exist in the merged dataframe we made. 

In [156]:
# return a cleaned list of author names 
r = re.compile("surname")
author_headers = (filter(r.match, data.columns ))
author_list = []
for header in author_headers:
    author_list += data[header].tolist()
    
cleanedList = [x for x in author_list if str(x) != 'nan']

# stylistically clean, but includes duplicates names
len(cleanedList)

273

Now, we can create a function that will iterate through the list of unique names we have. If a unique name is at least 80 percent similar to another name in the list, an alternate spelling will be added to the relevant surname key

In [166]:
def levenshtein(cleanedList):
    unique_names = {} 
    for i in cleanedList:
        if i not in unique_names.keys(): # if it's already in the list, skip it. 
            hold = process.extract(i, unique_names.keys(), limit=5)

            # we have not found a match yet, we're just getting started. 
            found = False
            for k in range(len(hold)):
                if hold[k][1] > 85 and not found:       # also make sure it's not found-- we don't want multiple matches. 
                    unique_names[hold[k][0]].append(i)  # append to the matching location... 
                                                        
                    # we found a match!
                    found = True    
            if not found: 
                # if we didn't find a match, add it to the list. 
                unique_names[i] = [i] 
    return unique_names

authors = levenshtein(cleanedList)


In this case, we can see that setting the threshold to 85 percent captures the "Faghiri/Faghiti" typeo, however, a threshold this low captures authors that were actually two separate ones. In this case, I would probably just recommend manually correcting this one particular entry. 

In [176]:
# manual correcting of typeo
merged_df['surname'] = merged_df['surname'].replace('Faghiti', 'Faghiri')
merged_df

,surname,firstname,Institution,Name,social,ID,Macroarea,Latitude,Longitude,Glottocode,...,Family,Subfamily,Genus,GenusIcon,ISO_codes,Samples_100,Samples_200,Country_ID,Source,Parent_ID
0,Sarvasy,Hannah,Western Sydney University,Nungon,(also Alba Tuninetti and other collaborators o...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fascinetto-Zago,Karina,Benemérita Universidad Autónoma de Puebla / Le...,Venetian,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Park,Kihyo,Cornell University,Korean,NaN,kor,Eurasia,37.5,128.0,kore1280,...,Korean,NaN,Korean,NaN,kor,True,True,KR KP,Chang-1996 Cho-1967 Gil-1991 Kim-1972 Kim-1986...,genus-korean
3,Park,Kihyo,Cornell University,Korean,NaN,family-korean,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Park,Kihyo,Cornell University,Korean,NaN,genus-korean,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,cBECC51,NaN,NaN,NaN,NaN,NaN,family-korean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Wong,Patrick,Chinese University of Hong Kong,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,Wray,Samantha,Dartmouth,Tagalog,NaN,tag,Papunesia,15.0,121.0,taga1270,...,Austronesian,NaN,Greater Central Philippine,NaN,tgl,True,True,PH,Aldridge-2004 Bloomfield-1917 De-Guzman-1978 E...,genus-greatercentralphilippine
297,Zhang,Miao,NaN,Changsha Xiang,@Miao_Zhang_dr,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,Zhao,Haoran,Leiden University,Chinese,NaN,genus-chinese,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,c33803F,NaN,NaN,NaN,NaN,NaN,family-sinotibetan


In [192]:
def get_cleaner_df(df):
    cleaner_df = df.loc[:, ('surname', 'firstname', 'Name', 'Institution', 'Family', 'social')]
    cleaner_df = cleaner_df.where(pd.notnull(subset_df), '')
    cleaner_df = df['Researcher'] = df['firstname'] + ' ' + df['surname']

    return cleaner_df

get_cleaner_df(merged_df)

0               Hannah Sarvasy
1      Karina  Fascinetto-Zago
2                   Kihyo Park
3                   Kihyo Park
4                   Kihyo Park
                ...           
295               Patrick Wong
296              Samantha Wray
297                 Miao Zhang
298                Haoran Zhao
299                        NaN
Length: 300, dtype: object

In [209]:
def get_cleaner_df(df):
    # add the first and last name to one cell for node plotting 
    df['Reseracher'] = df['firstname'] + ' ' + df['surname']
    
    # subset only the columns you want 
    subset_df = df.loc[:, ('surname','firstname','Name','Institution', 'Family', 'social', 'Researcher')]
    subset_df = subset_df.where(pd.notnull(subset_df), '')
    return subset_df
    
researchers_clean = get_cleaner_df(merged_df)

In [211]:
subset_df = researchers_clean.rename(columns={"Institution":"weights", 
                                              "Researcher":'sources', 
                                              'Name':"targets"})

In [223]:
# set the network parameters
net = Network(
    notebook=True,
    cdn_resources="remote",
    bgcolor="#222222",
    font_color="white",
    height="750px",
    width="100%",
)


# this parameters allows you to change the network once it's in HTML format
author_net.show_buttons(filter_=['physics'])

sources = subset_df['sources']
targets = subset_df['targets']
weights = subset_df['weights']
edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    author_net.add_node(src, src, title=src)
    author_net.add_node(dst, dst, title=dst)
    author_net.add_edge(src, dst, value=w)

neighbor_map = author_net.get_adj_list()
author_net.show("final_network.html")
 

final_network.html


AttributeError: 'NoneType' object has no attribute 'render'

In [93]:
G = nx.Graph()
G.add_node('Me')